<a href="https://colab.research.google.com/github/youcef-4/VisDrone/blob/main/tph_yolov5_on_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/cv516Buaa/tph-yolov5

Cloning into 'tph-yolov5'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 246 (delta 111), reused 178 (delta 76), pack-reused 0
Receiving objects: 100% (246/246), 9.26 MiB | 10.89 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [ ]:
!cd tph-yolov5


In [ ]:
!pip install -r /content/tph-yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python /content/tph-yolov5/VisDrone2YOLO_lable.py

548


In [ ]:
!python /content/tph-yolov5/val.py --weights /content/drive/MyDrive/VisDrone/yolov5l-xs-2.pt --img 1996 --data /content/tph-yolov5/data/VisDrone.yaml --augment --save-txt  --save-conf --task val --batch-size 8 --verbose --name v5l-xs

val: data=/content/tph-yolov5/data/VisDrone.yaml, weights=['/content/drive/MyDrive/VisDrone/yolov5l-xs-2.pt'], batch_size=8, imgsz=1996, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=True, verbose=True, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=tph-yolov5/runs/val, name=v5l-xs, exist_ok=False, half=False
YOLOv5 🚀 052dfeb torch 2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model Summary: 589 layers, 68030516 parameters, 0 gradients

100% 1.44G/1.44G [00:49<00:00, 31.5MB/s]
Unzipping ../datasets/VisDrone/VisDrone2019-DET-train.zip...
100% 77.9M/77.9M [00:04<00:00, 19.4MB/s]
Unzipping ../datasets/VisDrone/VisDrone2019-DET-val.zip...
100% 297M/297M [00:14<00:00, 21.8MB/s]
Unzipping ../datasets/VisDrone/VisDrone2019-DET-test-dev.zip...
100% 292M/292M [00:13<00:00, 22.2MB/s]
Unzipping ../datasets/VisDrone/VisDrone2019-DET-test-challenge.zip...
Converting ../datasets/VisDrone/VisDrone2019-DET-train: 6471it [00:37, 1

In [ ]:
!pip install "torch==1.7.0" -f https://download.pytorch.org/whl/torch_stable.html
!pip install "torchvision==0.8.1" -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.0 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+

In [ ]:
!wandb disabled
!python /content/tph-yolov5/train.py --img 1536 --adam --batch 3 --epochs 80 --data /content/tph-yolov5/data/VisDrone.yaml --weights yolov5l.pt --hy /content/tph-yolov5/data/hyps/hyp.VisDrone.yaml --cfg /content/tph-yolov5/models/yolov5l-xs-tph.yaml --name v5l-xs-tph
#!python /content/tph-yolov5/train.py --img 1536 --adam --batch 3 --epochs 80 --data /content/tph-yolov5/data/VisDrone.yaml --weights yolov5l.pt --hy /content/tph-yolov5/data/hyps/hyp.VisDrone.yaml --cfg /content/tph-yolov5/models/yolov5l-tph-plus.yaml --name v5l-tph-plus

W&B disabled.
2023-05-08 16:41:49.611652: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 16:41:52.188072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=yolov5l.pt, cfg=/content/tph-yolov5/models/yolov5l-xs-tph.yaml, data=/content/tph-yolov5/data/VisDrone.yaml, hyp=/content/tph-yolov5/data/hyps/hyp.VisDrone.yaml, epochs=80, batch_size=3, imgsz=1536, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=True, sync_bn=False, workers=8, project=tph-yolov5/runs/train, name=v5l-xs-tph, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=10